In [3]:
!pip install gensim==3.8.3
!pip install nltk==3.5

  Using cached gensim-3.8.3.tar.gz (23.4 MB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for gensim
  Running setup.py clean for gensim
Failed to build gensim
ERROR: Could not build wheels for gensim, which is required to install pyproject.toml-based projects


In [2]:
import pandas as pd
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [4]:
try:
    from google.colab import files
    !wget -P DATAPATH https://raw.githubusercontent.com/practical-nlp/practical-nlp/master/Ch4/Data/Sentiment%20and%20Emotion%20in%20Text/train_data.csv
    !wget -P DATAPATH https://raw.githubusercontent.com/practical-nlp/practical-nlp/master/Ch4/Data/Sentiment%20and%20Emotion%20in%20Text/test_data.csv
    !ls -lah DATAPATH
    filepath = "DATAPATH/train_data.csv"
except ModuleNotFoundError:
    filepath = "Data/Sentiment and Emotion in Text/train_data.csv"

--2023-12-06 23:57:55--  https://raw.githubusercontent.com/practical-nlp/practical-nlp/master/Ch4/Data/Sentiment%20and%20Emotion%20in%20Text/train_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2479133 (2.4M) [text/plain]
Saving to: ‘DATAPATH/train_data.csv’

train_data.csv      100%[===================>]   2.36M  --.-KB/s    in 0.08s   

2023-12-06 23:57:56 (28.6 MB/s) - ‘DATAPATH/train_data.csv’ saved [2479133/2479133]

--2023-12-06 23:57:56--  https://raw.githubusercontent.com/practical-nlp/practical-nlp/master/Ch4/Data/Sentiment%20and%20Emotion%20in%20Text/test_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubuser

In [5]:
df = pd.read_csv(filepath)
df.head()

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...


In [6]:
df['sentiment'].value_counts()

worry         7433
neutral       6340
sadness       4828
happiness     2986
love          2068
surprise      1613
hate          1187
fun           1088
relief        1021
empty          659
enthusiasm     522
boredom        157
anger           98
Name: sentiment, dtype: int64

In [8]:
shortlist = ['neutral', "happiness", "worry"]
df_subset = df[df['sentiment'].isin(shortlist)]
df_subset['sentiment'].value_counts()

worry        7433
neutral      6340
happiness    2986
Name: sentiment, dtype: int64

In [10]:
from nltk.tokenize import TweetTokenizer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

tweeter = TweetTokenizer(strip_handles=True,preserve_case=True)
mystopwords = set(stopwords.words("english"))
def preprocess_corpus(texts):
  def remove_stops_digits(tokens):
    return [token for token in tokens if token not in mystopwords and not token.isdigit()]
  return [remove_stops_digits(tweeter.tokenize(content)) for content in texts ]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [11]:
mydata = preprocess_corpus(df_subset['content'])
mycats = df_subset['sentiment']


In [12]:
len(mydata),len(mycats)

(16759, 16759)

In [15]:
from sklearn.model_selection import train_test_split
train_data, test_data, train_cats, test_cats  = train_test_split(mydata,mycats,random_state=1234)

In [17]:
train_doc2vec = [TaggedDocument((d), tags=[str(i)]) for i, d in enumerate(train_data)]
train_doc2vec[0:5]

[TaggedDocument(words=['I', "caaaaan't", 'sleep', '...', '3.30', '!', 'wahhhh', '...', 'wanna', 'cry'], tags=['0']),
 TaggedDocument(words=['based', 'future', 'AND', 'FORGETTING', '/', 'IGNORING', 'present', ',', 'best', 'keeper', 'according', 'dhoni', 'parthiv'], tags=['1']),
 TaggedDocument(words=['And', 'good', 'morning', '!', 'Up', 'early', 'bad', 'conscience', ',', 'trying', 'make', 'taking', 'day', 'yesterday', ',', '?', ':p'], tags=['2']),
 TaggedDocument(words=['hahaha', "chivalry's", 'dead', ',', 'rare'], tags=['3']),
 TaggedDocument(words=['joining', 'twitter'], tags=['4'])]

In [18]:
model = Doc2Vec(vector_size=50,alpha=0.025, min_count=5, dm =1, epochs=100)
model.build_vocab(train_doc2vec)
model.train(train_doc2vec, total_examples=model.corpus_count, epochs=model.epochs)
model.save('d2v.model')


In [21]:
model2 = Doc2Vec.load('d2v.model')
train_vectors  = [model2.infer_vector(tokens) for tokens in train_data]
test_vectors  = [model2.infer_vector(tokens) for tokens in test_data]
from sklearn.linear_model import LogisticRegression
myclass = LogisticRegression(class_weight = 'balanced')
myclass.fit(train_vectors, train_cats)

preds = myclass.predict(test_vectors)
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(test_cats, preds))

              precision    recall  f1-score   support

   happiness       0.32      0.52      0.39       713
     neutral       0.44      0.53      0.48      1595
       worry       0.59      0.35      0.44      1882

    accuracy                           0.45      4190
   macro avg       0.45      0.47      0.44      4190
weighted avg       0.49      0.45      0.45      4190

